
# Advanced Tutorial

This tutorial shows some more advanced features of the Data types presented in the [previous tutorial](./intermidiate.ipynb).

## Working with custom atomic names

Many quantum codes, and AiiDA itself, allow for defining custom atomic names. This can be usefull when defining some extra features, such as magnetic ordering or on-site Hubbard values. Nevertheless, `Phonopy` is not handling such cases, and this could be a bottleneck for the usage of the package in AiiDA. In `aiida-phonopy`, we manage to overcome this problem! Let's see how does it work!

### A structure with kinds

Let's define a very simple structure that contains two atoms of the same species. Let's take cubic silicon!

In [1]:
from local_module import load_temp_profile
from aiida.plugins import DataFactory

load_temp_profile(name="advanced-tutorial")

StructureData = DataFactory("core.structure")

a = 2.716
cell = [[0,a,a],[a,0,a],[a,a,0]]

structure = StructureData(cell=cell)
structure.append_atom(position=(a,a,a), symbols="Si", name="Si1")
structure.append_atom(position=(1.5*a,1.5*a,1.5*a), symbols="Si", name="Si2")

Now we can pass load it in the {py:class}`aiida_phonopy.data.preprocess.PreProcessData` with a supercell matrix of (2,2,2). Let's see if the supercell has the correct kinds.

In [2]:
PreProcessData = DataFactory("phonopy.preprocess")

preprocess_data =  PreProcessData(structure=structure, supercell_matrix=[2,2,2])

supercell = preprocess_data.get_supercell()
supercell.sites

[<Site: kind name 'Si1' @ 2.716,2.716,2.716>,
 <Site: kind name 'Si1' @ 2.716,5.432,5.432>,
 <Site: kind name 'Si1' @ 5.432,2.716,5.432>,
 <Site: kind name 'Si1' @ 5.432,5.432,8.148>,
 <Site: kind name 'Si1' @ 5.432,5.432,2.716>,
 <Site: kind name 'Si1' @ 5.432,8.148,5.432>,
 <Site: kind name 'Si1' @ 8.148,5.432,5.432>,
 <Site: kind name 'Si1' @ 8.148,8.148,8.148>,
 <Site: kind name 'Si2' @ 4.074,4.074,4.074>,
 <Site: kind name 'Si2' @ 4.074,6.79,6.79>,
 <Site: kind name 'Si2' @ 6.79,4.074,6.79>,
 <Site: kind name 'Si2' @ 6.79,6.79,9.506>,
 <Site: kind name 'Si2' @ 6.79,6.79,4.074>,
 <Site: kind name 'Si2' @ 6.79,9.506,6.79>,
 <Site: kind name 'Si2' @ 9.506,6.79,6.79>,
 <Site: kind name 'Si2' @ 9.506,9.506,9.506>]

Great! The supercell structure has the correct _kind names_.

If we have a look at the number of structure with displacements, we will notice they will be higher than the silicon structure with only chemical symbols.

In [3]:
supercells = preprocess_data.get_supercells_with_displacements()
print(f"Number of displacements: {len(supercells)}")

Number of displacements: 2


This may or may not be a wanted behaviour. If you want `Phonopy` to not distinguish atoms on their _name_, you can initialize the `PreProcessData` with the flag `distinguish_kinds = False`, as follows:

In [4]:
preprocess_data = PreProcessData(structure, supercell_matrix=[2,2,2], distinguish_kinds=False)
supercells = preprocess_data.get_supercells_with_displacements()
print(f"Now the number of displacements are: {len(supercells)}")

Now the number of displacements are: 1


```{admonition} Exercise
:class: tip
Verify that the supercell still have the same _kind names_.
```